In [4]:
import os
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
import glob
from Taxonomy.TaxonomyParser import TaxonomyParser
pd.set_option("display.max_rows",50)

In [23]:
folder_path= "data/adsData"
file_list = glob.glob(f"{folder_path}/*_AlNajm.csv")

df = pd.concat([pd.read_csv(file) for file in file_list], ignore_index=True)
df2= pd.read_csv("data/savedData/datacleaned.csv")

In [6]:
df.head()

,Date,Campaign_Name,Platform,Impressions,Clicks,Conversions,Spend_USD
0,2023-02-01,AN_Intl_Loyalty_QAT_AR_Leisure_Video_Feb_NA,DV360,28143,3807,138,2624.99
1,2023-02-02,AN_Intl_Loyalty_QAT_AR_Leisure_Video_Feb_NA,DV360,49638,5908,324,6309.40
2,2023-02-03,AN_Intl_Loyalty_QAT_AR_Leisure_Video_Feb_NA,DV360,23459,2340,185,2021.46
3,2023-02-04,AN_Intl_Loyalty_QAT_AR_Leisure_Video_Feb_NA,DV360,11104,356,20,517.01
4,2023-02-05,AN_Intl_Loyalty_QAT_AR_Leisure_Video_Feb_NA,DV360,14296,2017,119,2453.88


In [8]:
def test_taxonomy_parser_from_df(df):

    parser = TaxonomyParser()
    parsed_df = parser.parse_dataframe(df)

    # Flag invalid routes (e.g., "NA" or bad format)
    parsed_df["Invalid_Route_Flag"] = ~parsed_df["Route"].str.match(r"^[A-Z]{3}-[A-Z]{3}$")

    print("Parsed sample:")
    display(parsed_df.head())  # for Jupyter
    return parsed_df


In [9]:
parsed_df = test_taxonomy_parser_from_df(df)


Parsed sample:


,Date,Campaign_Name,Platform,Impressions,Clicks,Conversions,Spend_USD,BusinessUnit,Objective,Market,Language,Audience,Channel,Month,Route,Validity,Invalid_Route_Flag
0,2023-02-01,AN_Intl_Loyalty_QAT_AR_Leisure_Video_Feb_NA,DV360,28143,3807,138,2624.99,Intl,Loyalty,QAT,AR,Leisure,Video,Feb,NA,False,True
1,2023-02-02,AN_Intl_Loyalty_QAT_AR_Leisure_Video_Feb_NA,DV360,49638,5908,324,6309.40,Intl,Loyalty,QAT,AR,Leisure,Video,Feb,NA,False,True
2,2023-02-03,AN_Intl_Loyalty_QAT_AR_Leisure_Video_Feb_NA,DV360,23459,2340,185,2021.46,Intl,Loyalty,QAT,AR,Leisure,Video,Feb,NA,False,True
3,2023-02-04,AN_Intl_Loyalty_QAT_AR_Leisure_Video_Feb_NA,DV360,11104,356,20,517.01,Intl,Loyalty,QAT,AR,Leisure,Video,Feb,NA,False,True
4,2023-02-05,AN_Intl_Loyalty_QAT_AR_Leisure_Video_Feb_NA,DV360,14296,2017,119,2453.88,Intl,Loyalty,QAT,AR,Leisure,Video,Feb,NA,False,True


In [10]:
parsed_df["Validity"].value_counts()

Validity
False    29240
True     14620
Name: count, dtype: int64

In [11]:
def join_campaign_with_budget_df(campaign_df: pd.DataFrame, budget_df: pd.DataFrame) -> pd.DataFrame:
    def normalize_route(val):
        if pd.isna(val) or str(val).lower() == 'nan':
            return 'NA'
        return val

    campaign_df = campaign_df.copy()
    budget_df = budget_df.copy()

    campaign_df["Route"] = campaign_df["Route"].apply(normalize_route)
    budget_df["Route"] = budget_df["Route"].apply(normalize_route)

    campaign_df["Date"] = pd.to_datetime(campaign_df["Date"])
    budget_df["Date"] = pd.to_datetime(budget_df["Date"])

    join_keys = ["BusinessUnit", "Objective", "Market", "Language", "Audience", "Channel", "Month", "Route", "Date"]
    
    merged_df = pd.merge(
        campaign_df,
        budget_df[join_keys + ["Planned_Daily_Budget_USD"]],
        how="left",
        on=join_keys
    )

    return merged_df


In [12]:
bdgt= pd.read_csv("data/Budget_Pacing.csv")

/tmp/ipykernel_1220385/2971148265.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  bdgt= pd.read_csv("data/Budget_Pacing.csv")


In [13]:
joined_df = join_campaign_with_budget_df(parsed_df, bdgt)


In [17]:
validdf= joined_df[joined_df["Validity"]==True]

In [22]:
validdf.to_csv("validdf.csv")

In [25]:
df= df2.copy()
df['Date'] = pd.to_datetime(df['Date']) 

df_last_week = df[
    (df['Date'] >= pd.Timestamp('2025-01-24')) &
    (df['Date'] <= pd.Timestamp('2025-01-31'))
]


In [27]:
df_last_week.to_csv("lastweekdf.csv")

In [29]:
df_last_week["Date"].unique()

<DatetimeArray>
['2025-01-26 00:00:00', '2025-01-27 00:00:00', '2025-01-30 00:00:00',
 '2025-01-28 00:00:00', '2025-01-29 00:00:00', '2025-01-31 00:00:00',
 '2025-01-25 00:00:00', '2025-01-24 00:00:00']
Length: 8, dtype: datetime64[ns]